# Extracting Density Functional Theory (DFT) Predictions of Interfacial Tension

Using Sriteja Mantha's modification of Huikuan Chao's code for calculating the interfacial tension along a planar surface between a polyol-rich phase and a CO2-rich phase, I computed DFT predictions of interfacial tension. I need to reformat them to compare them more easily with experimental measurements using Prof. Di Maio's G-ADSA technique. This notebook performs that reformatting and validates it.

**Note:** the pressure given by the DFT results can be substantially different from the pressure used in PC-SAFT and provided in the `blkrho.inp` document. As a result, I interpolate the PC-SAFT data using the pressures from the DFT results so that the solubility data are given at the same pressure as the interfacial tension data.

## Sensitivity Analysis 

I am expanding my simulations to study the sensitivity of interfacial tension to small variations in the PC-SAFT parameters, $\sigma$, $\epsilon$, $N$, and the temperature $T$. This investigation requires a different folder structure, so I have modified the data formatting notebook `extract_dft_if_tension.ipynb` accordingly.

In [8]:
import numpy as np
import os
import glob
import pandas as pd

import dftproc

from importlib import reload

reload(dftproc)

# path to folder containing folders of DFT predictions
dft_pred_folder = '../../../Wang/dft_string_method/bi_surf/3k2f_31c_sensitivity/' #3k2f_31c_sensitivity//' #N2f/60c/' # 3k2f_60c_sensitivity/' '../../../Wang/dft_string_method/bi_surf/' # 
# path to folder of saved results
save_folder = 'dft_pred/3k2f_31c_sensitivity/' #3k2f_31c_sensitivity//' #'N2f/60c/' #3k2f_60c_sensitivity/' #  'dft_pred/' # 'dft_pred/3k2f_60c_sensitivity/' #
# code for filenames
folder_code = '*' #'*k*f*c*' # '*' #

# CONVERSIONS
Pa_2_kPa = 0.001 # converts Pa to kPa
Nm_2_mNm = 1000 # converts N/m to mN/m
MPa_2_kPa = 1000 # converts kPa to MPa

def parse_folder_name(folder):
    """Parses folder name to create filename of structure #k#f_#c (Mw, functionality, temp. in C)."""
    return folder.split('\\')[-1] # extracts filename from filepath

In [9]:
# collects list of folders of interfacial tension predictions from DFT
folder_list_temp = os.path.join(dft_pred_folder + folder_code)
folder_list = glob.glob(folder_list_temp)

# processes data in each folder to produce a file of interfacial tension data
for folder in folder_list:
    # loads list of files in the folder with surface tension data
    file_list_temp = os.path.join(folder + '\\*surftsn.dat')
    file_list = glob.glob(file_list_temp)
    # skips empty folders and files
    if len(file_list) == 0:
        continue
    # initializes lists of data
    p = []
    if_tension = []
    for file in file_list:
        data = np.genfromtxt(file)
        p += [Pa_2_kPa*data[-1,0]] # extracts pressure [kPa]
        if_tension += [Nm_2_mNm*data[-1,1]] # extracts interfacial tension [N/m]
    
    # computes solubility [w/w] and corresponding pressure [MPa]
    p_dft, props = dftproc.get_props(folder + '//' + 'blkrho.inp', sort=True)
    rho_co2, solub, spec_vol = props
    
    # interpolates solubility (pressure is converted to kPa)
    solub_interp = np.interp(p, p_dft*MPa_2_kPa, solub)
    # interpolates specific volume (pressure is converted to kPa)
    spec_vol_interp = np.interp(p, p_dft*MPa_2_kPa, spec_vol)
    # interpolates specific volume (pressure is converted to kPa)
    rho_co2_interp = np.interp(p, p_dft*MPa_2_kPa, rho_co2)
    
    # orders predicted data points in terms of increasing pressure
    data = list(zip(p, if_tension, solub_interp, spec_vol_interp, rho_co2_interp))
    data.sort(key=lambda x : x[0])
    
    # saves DFT predictions in a file
    df = pd.DataFrame(data, columns=['p actual [kPa]', 'if tension [mN/m]', 
                                     'solubility [w/w]', 'specific volume (fit) [mL/g]', 'co2 density [g/mL]'])
    # parses folder name to create new file name (.csv)
    filename = parse_folder_name(folder) + '.csv'
    print(filename)
    df.to_csv(save_folder + filename, index=False)

epsn_217-0.csv
epsn_224-0.csv
epsn_224-0~sigma_3-07.csv
epsn_228-5~sigma_3-01~N_144.csv
epsn_232-8~sigma_3-22.csv
epsn_240-0.csv
epsn_240-0~sigma_3-07.csv
epsn_258-0~sigma_3-17.csv
N_117-0.csv
N_129-0.csv
sigma_2-86.csv
sigma_3-07.csv
sigma_3-16.csv
T_303-75.csv
T_304-75.csv
